In [1]:
import pandas as pd
import numpy as np
import os
from config import *

In [2]:
csvfile = "./decorated_data.csv"
df = pd.read_csv(csvfile,dtype=object)

In [3]:
df.head()

,ticker,cusip,indsec,indgrp
0,COP,20825C104,Energy,Integrated oil Companies
1,JNJ,478160104,Health Care,Major Pharmaceuticals
2,NaN,58441K100,NaN,NaN
3,ESRX,30219G108,Health Care,Medical/Nursing Services
4,SATS,25470M109,Technology,Radio And Television Broadcasting And Communic...


In [4]:
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [5]:
# create an engine to postgresql db
user = config['psql_user']
password = config['psql_pwd']
host = 'localhost'
port = '5432'
db = config['psql_db']
url = 'postgresql://{}:{}@{}:{}/{}'
url = url.format(user, password, host, port, db)

# The return value of create_engine() is our connection object
engine = sqlalchemy.create_engine(url, client_encoding='utf8')

In [6]:
# Create a connection to the engine called conn
conn = engine.connect()

In [7]:
# Use `declarative_base` from SQLAlchemy to connect your class to your PostgreSQL database
Base = declarative_base()

In [8]:
class SecuritiesEx(Base):
    __tablename__ = 'securitiesex'
    id = Column(Integer, primary_key=True)
    ticker = Column(Text)
    cusip = Column(Text)
    indsec = Column(Text)
    indgrp = Column(Text)
    
    def __repr__(self):
        return f"indsec={self.indsec}, indgrp={self.indgrp}, cusip={self.cusip} ticker={self.ticker}"
    

In [9]:
# Use `create_all` to create the indsectorindgroup table in the database
Base.metadata.create_all(engine)

In [10]:
# Use MetaData from SQLAlchemy to reflect the tables\n",
metadata = MetaData(bind=engine)
metadata.reflect()

In [11]:
# Save the reference to the `securitiesex` table as a variable called `table`
table = sqlalchemy.Table('securitiesex', metadata, autoload=True)

In [12]:
# Use `table.delete()` to remove any existing data.
conn.execute(table.delete())

In [13]:
data = df.to_dict(orient='records')
print(data)

[{'ticker': 'COP', 'cusip': '20825C104', 'indsec': 'Energy', 'indgrp': 'Integrated oil Companies'}, {'ticker': 'JNJ', 'cusip': '478160104', 'indsec': 'Health Care', 'indgrp': 'Major Pharmaceuticals'}, {'ticker': nan, 'cusip': '58441K100', 'indsec': nan, 'indgrp': nan}, {'ticker': 'ESRX', 'cusip': '30219G108', 'indsec': 'Health Care', 'indgrp': 'Medical/Nursing Services'}, {'ticker': 'SATS', 'cusip': '25470M109', 'indsec': 'Technology', 'indgrp': 'Radio And Television Broadcasting And Communications Equipment'}, {'ticker': 'VZ', 'cusip': '92343V104', 'indsec': 'Public Utilities', 'indgrp': 'Telecommunications Equipment'}, {'ticker': nan, 'cusip': '531229607', 'indsec': 'None', 'indgrp': 'None'}, {'ticker': 'DVA', 'cusip': '23918K108', 'indsec': 'Health Care', 'indgrp': 'Hospital/Nursing Management'}, {'ticker': 'DE', 'cusip': '244199105', 'indsec': 'Capital Goods', 'indgrp': 'Industrial Machinery/Components'}, {'ticker': 'USG', 'cusip': '903293405', 'indsec': 'Capital Goods', 'indgrp': 

In [14]:

conn.execute(table.insert(), data)

In [15]:
conn.execute("select * from securitiesex").fetchall()

[(1, 'COP', '20825C104', 'Energy', 'Integrated oil Companies'),
 (2, 'JNJ', '478160104', 'Health Care', 'Major Pharmaceuticals'),
 (3, 'NaN', '58441K100', 'NaN', 'NaN'),
 (4, 'ESRX', '30219G108', 'Health Care', 'Medical/Nursing Services'),
 (5, 'SATS', '25470M109', 'Technology', 'Radio And Television Broadcasting And Communications Equipment'),
 (6, 'VZ', '92343V104', 'Public Utilities', 'Telecommunications Equipment'),
 (7, 'NaN', '531229607', 'None', 'None'),
 (8, 'DVA', '23918K108', 'Health Care', 'Hospital/Nursing Management'),
 (9, 'DE', '244199105', 'Capital Goods', 'Industrial Machinery/Components'),
 (10, 'USG', '903293405', 'Capital Goods', 'Building Materials'),
 (11, 'STOR', '862121100', 'Consumer Services', 'Real Estate Investment Trusts'),
 (12, 'PSX', '718546104', 'Energy', 'Integrated oil Companies'),
 (13, 'VRSK', '92345Y106', 'Technology', 'EDP Services'),
 (14, 'CBI', '167250109', 'Basic Industries', 'Engineering & Construction'),
 (15, 'MON', '61166W101', 'Basic Indu

In [16]:
conn.close()